In [1]:
# import dicom, cv2, re, sys
import os, fnmatch, shutil, subprocess
import numpy as np
from PIL import Image
from IPython.utils import io
import numpy as np
np.random.seed(1234)
import matplotlib as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore') # we ignore a RuntimeWarning produced from dividing by zero
import os, sys, urllib, gzip
import cPickle as pickle
sys.setrecursionlimit(10000)
import glob
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from IPython.display import Image as IPImage

from PIL import Image
from lasagne.layers import get_output, InputLayer, DenseLayer, Upscale2DLayer, ReshapeLayer
from lasagne.init import GlorotUniform
from lasagne.nonlinearities import rectify, leaky_rectify, tanh, sigmoid, softmax
from lasagne.updates import nesterov_momentum, adam
from lasagne.objectives import categorical_crossentropy, binary_crossentropy
from nolearn.lasagne import NeuralNet, BatchIterator, PrintLayerInfo
from lasagne.layers import Conv2DLayer as Conv2DLayer
from lasagne.layers import MaxPool2DLayer as MaxPool2DLayer
import theano 
import theano.tensor as T
import lasagne
import time
try:
    from lasagne.layers.dnn import Conv2DDNNLayer as Conv2DLayer
    from lasagne.layers.dnn import Conv3DDNNLayer as Conv3DLayer
    from lasagne.layers.dnn import MaxPool2DDNNLayer as MaxPool2DLayer
    print 'Using cuda_convnet (faster)'
except ImportError:
    from lasagne.layers import Conv2DLayer as Conv2DLayer
    from lasagne.layers import Conv3DLayer as Conv3DLayer
    from lasagne.layers import Pool3Layer as Pool3Layer
    from lasagne.layers import ReshapeLayer
    print 'Using lasagne.layers (slower)'
    
import theano
import theano.tensor as T

from lasagne.layers import Layer
from lasagne import init
from lasagne import nonlinearities
from scipy.misc import imresize, imread
from PIL import ImageOps
import scipy as sp
import scipy.ndimage.morphology
from skimage.morphology import convex_hull_image
from skimage.restoration import denoise_tv_chambolle, denoise_bilateral
import matplotlib.cm as cm
from scipy.optimize import minimize
from math import floor

Using lasagne.layers (slower)


In [2]:
from batchNormalization import BatchNormLayer

def batch_norm(layer, **kwargs):
    nonlinearity = getattr(layer, 'nonlinearity', None)
    if nonlinearity is not None:
        layer.nonlinearity = nonlinearities.identity
    if hasattr(layer, 'b') and layer.b is not None:
        del layer.params[layer.b]
        layer.b = None
    layer = BatchNormLayer(layer, **kwargs)
    if nonlinearity is not None:
        from lasagne.layers import NonlinearityLayer
        layer = NonlinearityLayer(layer, nonlinearity)
    return layer

In [3]:
X_train = np.load("X_train_step12.npy")
y_train = np.load("y_train_step12.npy")
setNames_train = np.load("setNames_train_step12.npy")

In [4]:
L,W,H = X_train.shape[1::]
X_train = X_train.reshape(-1,1,L,W,H).astype("float32")
y_train = y_train.astype("float32")
print X_train.shape

(630, 1, 5, 225, 225)


In [5]:
gate_parameters = lasagne.layers.recurrent.Gate(
    W_in=lasagne.init.Orthogonal(), W_hid=lasagne.init.Orthogonal(),
    b=lasagne.init.Constant(0.))
cell_parameters = lasagne.layers.recurrent.Gate(
    W_in=lasagne.init.Orthogonal(), W_hid=lasagne.init.Orthogonal(),
    # Setting W_cell to None denotes that no cell connection will be used.
    W_cell=None, b=lasagne.init.Constant(0.),
    # By convention, the cell nonlinearity is tanh in an LSTM.
    nonlinearity=lasagne.nonlinearities.tanh)

batch_size = 2

def build_lstm_cnn(input_var=None):
    
    
    conv_num_filters1 = 4
    conv_num_filters2 = 8
    conv_num_filters3 = 8
    filter_size1 = 7
    filter_size2 = 3
    filter_size3 = 3
    pool_size = 2
    pool_size2 = 4
    pad_in = 'valid'
    pad_out = 'full'
    
    # Input layer, as usual:                                                                                                                                                                                
    network = InputLayer(shape=(batch_size,1,L,W,H),input_var=input_var,name="input_layer")                                                                                                                             
        
    network = batch_norm(Conv3DLayer(
            network, num_filters=conv_num_filters1, filter_size=(1,filter_size1, filter_size1),
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform(),name="conv3_1"))
    
    network = Pool3Layer(network, pool_size=(1,pool_size, pool_size),name="pool3_1")
    
    network = batch_norm(Conv3DLayer(
            network, num_filters=conv_num_filters2, filter_size=(1,filter_size2, filter_size2),
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform(),name="conv3_2"))
    
    network = Pool3Layer(network, pool_size=(1,pool_size, pool_size),name="pool3_2")
    
    network = batch_norm(Conv3DLayer(
            network, num_filters=conv_num_filters3, filter_size=(1,filter_size3, filter_size3),
            nonlinearity=lasagne.nonlinearities.rectify,
            W=lasagne.init.GlorotUniform(),name="conv3_3"))
    
    network = Pool3Layer(network, pool_size=(1,pool_size2, pool_size2),name="pool3_3")
    
    Netshape = network.output_shape

    network = ReshapeLayer(network, shape = (-1,Netshape[2],Netshape[1],Netshape[3],Netshape[4]),name="reshape")
    
    # Our LSTM will have 10 hidden/cell units
    N_HIDDEN = 20
    GRAD_CLIP = 100.0
    network = lasagne.layers.recurrent.LSTMLayer(
    network, N_HIDDEN, ingate=gate_parameters, forgetgate=gate_parameters,
     cell=cell_parameters, outgate=gate_parameters,
    grad_clipping=GRAD_CLIP,
    learn_init=True,name="lstm_1")

    network = lasagne.layers.recurrent.LSTMLayer(
    network, N_HIDDEN, ingate=gate_parameters, forgetgate=gate_parameters,
     cell=cell_parameters, outgate=gate_parameters,
    grad_clipping=GRAD_CLIP,
    learn_init=True,name = "lstm_2")

    network = lasagne.layers.SliceLayer(network, -1, 1,name="slice")
    
    network = batch_norm(lasagne.layers.DenseLayer(network, num_units=1, \
                                  W = lasagne.init.Normal(), \
                                  nonlinearity=lasagne.nonlinearities.softmax,name = "dense"))
    
    return network

Now lets build a net and make sure we have the dimensions right:

In [6]:
network = build_lstm_cnn()

laylist = lasagne.layers.get_all_layers(network)
for l in laylist:
    print(l.name, lasagne.layers.get_output_shape(l))

num_params = lasagne.layers.count_params(network)
print("# of parameters is {}".format(num_params))

('input_layer', (2, 1, 5, 225, 225))
('conv3_1', (2, 4, 5, 219, 219))
(None, (2, 4, 5, 219, 219))
(None, (2, 4, 5, 219, 219))
('pool3_1', (2, 4, 5, 109, 109))
('conv3_2', (2, 8, 5, 107, 107))
(None, (2, 8, 5, 107, 107))
(None, (2, 8, 5, 107, 107))
('pool3_2', (2, 8, 5, 53, 53))
('conv3_3', (2, 8, 5, 51, 51))
(None, (2, 8, 5, 51, 51))
(None, (2, 8, 5, 51, 51))
('pool3_3', (2, 8, 5, 12, 12))
('reshape', (2, 5, 8, 12, 12))
('lstm_1', (2, 5, 20))
('lstm_2', (2, 5, 20))
('slice', (2, 20))
('dense', (2, 1))
(None, (2, 1))
(None, (2, 1))
# of parameters is 98484


In [7]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

def iterator(X, batchsize):
    indices = np.arange(len(X))
    for i in range(0, len(X) - batchsize + 1, batchsize):
        sli = indices[i:i+batchsize]
        yield X[sli]

def save_params(model, fn):
    with open(fn, 'w') as wr:
        pickle.dump(lasagne.layers.get_all_param_values(model), wr)

In [8]:
# Symbolic variable for the target network output.
# It will be of shape n_batch, because there's only 1 target value per sequence.

target_values = T.vector('target_output')
dtensor5 = T.TensorType('float32', (False,)*5)
input_var = dtensor5('inputs')

network = build_lstm_cnn(input_var)
# lasagne.layers.get_output produces an expression for the output of the net
network_output = lasagne.layers.get_output(network)

laylist = lasagne.layers.get_all_layers(network)
    
for l in laylist:
    print(l.name, lasagne.layers.get_output_shape(l))

num_params = lasagne.layers.count_params(network)

print("number of parameters is {}".format(num_params))
# The value we care about is the final value produced for each sequence
# so we simply slice it out.
predicted_values = network_output[:, -1]
# Our cost will be mean-squared error
# cost = T.mean((predicted_values - target_values)**2)
cost = lasagne.objectives.squared_error(predicted_values, target_values)
cost = cost.mean()

# Retrieve all parameters from the network
all_params = lasagne.layers.get_all_params(network ,trainable = True)
# Compute adam updates for training
updates = lasagne.updates.adam(cost, all_params)
# Theano functions for training and computing cost
train = theano.function([input_var, target_values],cost, updates=updates)
compute_cost = theano.function([input_var, target_values], cost)

('input_layer', (2, 1, 5, 225, 225))
('conv3_1', (2, 4, 5, 219, 219))
(None, (2, 4, 5, 219, 219))
(None, (2, 4, 5, 219, 219))
('pool3_1', (2, 4, 5, 109, 109))
('conv3_2', (2, 8, 5, 107, 107))
(None, (2, 8, 5, 107, 107))
(None, (2, 8, 5, 107, 107))
('pool3_2', (2, 8, 5, 53, 53))
('conv3_3', (2, 8, 5, 51, 51))
(None, (2, 8, 5, 51, 51))
(None, (2, 8, 5, 51, 51))
('pool3_3', (2, 8, 5, 12, 12))
('reshape', (2, 5, 8, 12, 12))
('lstm_1', (2, 5, 20))
('lstm_2', (2, 5, 20))
('slice', (2, 20))
('dense', (2, 1))
(None, (2, 1))
(None, (2, 1))
number of parameters is 98484


In [9]:
# Finally, launch the training loop.
print("Starting training...")
# We iterate over epochs: 
num_epochs = 10
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, batch_size, shuffle=True):
        inputs, targets = batch
        print inputs.shape, targets.shape
        
        train_err += train(inputs, targets)
        break
#         train_batches += 1
#             print train_fn2(inputs, targets)
    # And a full pass over the validation data:
#     val_err = 0
#     val_batches = 0
#     for batch in iterate_minibatches(X_val, y_val, batch_size, shuffle=False):
#         inputs, targets = batch
#         err = compute_cost(inputs, targets)
#         val_err += err[0]
#         val_batches += 1

    # Then we print the results for this epoch:                                                                                                                                                         
    print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
#     print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
#     print("  ratio:\t\t{:.6f}".format((train_err / train_batches)/(val_err / val_batches)))

Starting training...
(2, 1, 5, 225, 225) (2,)
i0 is 1, i1 is 3


NotImplementedError: is this allowed?
Apply node that caused the error: DiagonalSubtensor{inplace}(Reshape{6}.0, TensorConstant{1}, TensorConstant{3})
Toposort index: 153
Inputs types: [TensorType(float32, 6D), TensorType(int8, scalar), TensorType(int8, scalar)]
Inputs shapes: [(2, 5, 4, 7, 225, 219), (), ()]
Inputs strides: [(27594000, 5518800, 1379700, 197100, 876, 4), (), ()]
Inputs values: ['not shown', array(1, dtype=int8), array(3, dtype=int8)]
Outputs clients: [[Sum{axis=[3], acc_dtype=float64}(DiagonalSubtensor{inplace}.0), Shape_i{1}(DiagonalSubtensor{inplace}.0), Shape_i{4}(DiagonalSubtensor{inplace}.0), Shape_i{5}(DiagonalSubtensor{inplace}.0), Shape_i{2}(DiagonalSubtensor{inplace}.0), Shape_i{0}(DiagonalSubtensor{inplace}.0), Shape_i{3}(DiagonalSubtensor{inplace}.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "/usr/local/lib/python2.7/site-packages/ipykernel/ipkernel.py", line 175, in do_execute
    shell.run_cell(code,